### ESTO ES SOLO PARA CARGAR LOS DOCUMENTOS A UNA BASE DE SQLITE
Vi que hay varios segmentos que literalmente solo tienen titulos asi que me la rifo y crearé una buena base de datos

In [1]:
#Vector store
import sqlite3
from langchain_community.vectorstores import SQLiteVec
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document


# llms
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

from typing_extensions import List
from dotenv import load_dotenv
from pathlib import Path

import os

load_dotenv()
#

base_dir = Path.cwd()
leyes_dir = base_dir / "leyes"  # Adjust this path based on your actual directory structure

# If leyes_dir doesn't exist, try going up one level
if not leyes_dir.exists():
    leyes_dir = base_dir.parent / "leyes"


Base

In [2]:
import sqlite_vec

db_path = base_dir / "Database/leyes.db"
db_path.parent.mkdir(parents=True, exist_ok=True) 

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vector_store = SQLiteVec(
    table="leyes",
    connection=None,
    embedding=embeddings,
    db_file=db_path,
)   

Cargar Documentos con OpenAI Embeddings y Segmentacion semantica FUCK IT

In [3]:
import json
from pathlib import Path

# Load categories dictionary
with open(Path.cwd() / "categorias_sinonimos_bolivia.json", "r", encoding="utf-8") as f:
    categories_dict = json.load(f)

def categorize_text(text):
    """Tag text with categories based on term matches."""
    text_lower = text.lower()
    matched_categories = set()
    
    for category, terms in categories_dict.items():
        for term in terms:
            if term.lower() in text_lower:
                matched_categories.add(category)
                break
    
    return list(matched_categories)

def split_doc(file_path: str) -> List[Document]:
    # Read the file to extract the title (first line)
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        document_title = first_line if first_line else os.path.basename(file_path)
    
    # Continue with your existing loading process
    loader = TextLoader(file_path, encoding='utf-8')
    documento = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documento)
    
    # Add category tags and document title to metadata while preserving existing metadata
    for doc in docs:
        categories = categorize_text(doc.page_content)
        # Ensure metadata exists
        if doc.metadata is None:
            doc.metadata = {}
        
        # Preserve the existing metadata and add the categories field
        doc.metadata["categories"] = categories
        
        # Add document title to metadata
        doc.metadata["document_title"] = document_title
        
        # Add source file info if not already present
        if "source" not in doc.metadata:
            doc.metadata["source"] = os.path.basename(file_path)
    
    return docs

def add_doc_to_vector_db(vector_store: SQLiteVec, docs: List[Document]):
    vector_store.add_documents(docs)
    
# Load all laws from the directory
for file in os.listdir(leyes_dir):
    file_path = os.path.join(leyes_dir, file)
    print(f"Adding {file_path} to the vector store...")
    if os.path.isfile(file_path) and file.endswith('.txt'):
        docs = split_doc(file_path)
        add_doc_to_vector_db(vector_store, docs)

Adding e:\Code\Python\Asistente-Legal-API\App\leyes\1973-BO-COD-DL10135.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\1978-BO-RE-RS187444(1).txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2006-BO-DS-28710.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2008-BO-L-3988.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2010-BO-DS-N420.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2011-BO-L-N145.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2014-BO-DS-N2079.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2017-BO-DS-N3045.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2022-BO-DS-N4740.txt to the vector store...
Adding e:\Code\Python\Asistente-Legal-API\App\leyes\2022-BO-DS-N4780.txt to the vector store...
Adding e:\Code\Python\Asistente-Lega